# Counting All Things DPLA

Picking up from [part one](https://github.com/dchud/notebooks/blob/master/20160229-counting-things-in-dpla-with-sparksql.ipynb), let's 'count some more stuff.  This work continues to be inspired by [Corey Harper](https://open.library.ubc.ca/cIRcle/collections/55474/items/1.0220818) and [Mark Phillips](http://vphill.com/journal/post/5899/) and a lengthy conversation we had last summer.  My short term goal is to demonstrate an approach to help them move more quickly.

We will start again from recent (April 2016) [DPLA Bulk Downloads](http://dp.la/info/developers/download/?prefix=2016/04/), but this time we'll reach for a larger set, *all* the records from the recent export.  Because that's what Mark and Corey are doing.

In [5]:
!wget https://dpla-provider-export.s3.amazonaws.com/2016/04/all.json.gz

--2016-04-11 19:28:29--  https://dpla-provider-export.s3.amazonaws.com/2016/04/all.json.gz
Resolving dpla-provider-export.s3.amazonaws.com (dpla-provider-export.s3.amazonaws.com)... 54.231.18.169
Connecting to dpla-provider-export.s3.amazonaws.com (dpla-provider-export.s3.amazonaws.com)|54.231.18.169|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5094571507 (4.7G) [application/gzip]
Saving to: ‘all.json.gz’

100%[====================================>] 5,094,571,507 44.1MB/s   in 1m 55s 

2016-04-11 19:30:25 (42.2 MB/s) - ‘all.json.gz’ saved [5094571507/5094571507]



In [ ]:
!zcat all.json.gz | split -b 64M

In [8]:
raw_in = sc.textFile('x*') \
    .map(lambda line: line[1:] if line.startswith(',') and len(line) > 1 else line) \
    .filter(lambda line: len(line) > 0 and not line[0] in ['[', ']'])

In [9]:
json_in = sqlContext.read.json(raw_in)

In [10]:
json_in.write.save('all.parquet')

In [11]:
pq_in = sqlContext.read.load('all.parquet')

In [12]:
pq_in.registerTempTable('pq')

In [13]:
count = sqlContext.sql("SELECT COUNT(*) AS the_count FROM pq")
%time count.show()

+---------+
|the_count|
+---------+
| 11788875|
+---------+

CPU times: user 2.06 ms, sys: 201 µs, total: 2.26 ms
Wall time: 8.55 s


In [14]:
types = sqlContext.sql("""
SELECT `_source`.sourceResource.type AS type, COUNT(*) AS the_count 
FROM pq 
GROUP BY `_source`.sourceResource.type
ORDER BY the_count DESC
""")
%time types.show()

+--------------------+---------+
|                type|the_count|
+--------------------+---------+
|               image|  5705394|
|                text|  5156042|
|                null|   868174|
|        moving image|    27832|
|               sound|    14456|
|     physical object|     6720|
|           ["image"]|     6380|
|    ["image","text"]|     2689|
| ["physical object"]|     1109|
|["physical object...|       76|
|          collection|        2|
|            ["text"]|        1|
+--------------------+---------+

CPU times: user 19.5 ms, sys: 1.89 ms, total: 21.3 ms
Wall time: 1min 40s


In [15]:
types = pq_in.map(lambda rec: (rec['_source']['provider']['name'], rec['_source']['sourceResource']['type']))

In [16]:
types_count = types.map(lambda x: (x[0], 'None' if x[1] == None else x[1])) \
    .flatMap(lambda x: [(x[0], x[1])] if not '[' in x[1] else [(x[0], t) for t in eval(x[1])]) \
    .map(lambda x: (x, 1)) \
    .reduceByKey(lambda x, y: x + y)

In [17]:
%time types_count.collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 6 in stage 14.0 failed 1 times, most recent failure: Lost task 6.0 in stage 14.0 (TID 11322, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/ubuntu/spark-1.6.1-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/home/ubuntu/spark-1.6.1-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/ubuntu/spark-1.6.1-bin-hadoop2.6/python/pyspark/rdd.py", line 2346, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/ubuntu/spark-1.6.1-bin-hadoop2.6/python/pyspark/rdd.py", line 2346, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/ubuntu/spark-1.6.1-bin-ha